In [1]:
# import the necessary libraries
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks

In [2]:
# read the csv file in chunks of 100000 since it is very large with over 2 million rows
tp = pd.read_csv('covidcdc.csv', iterator=True, chunksize=100000)

In [3]:
# concatenate the dataset into a single dataframe
covid = pd.concat(tp, ignore_index=True)

C:\Users\tkh7_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
covid.head()

,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn
0,2020/06/28,2020/06/28,NaN,NaN,Laboratory-confirmed case,Unknown,0 - 9 Years,"Black, Non-Hispanic",Missing,Missing,No,Missing
1,2020/07/26,2020/07/26,NaN,NaN,Laboratory-confirmed case,Unknown,0 - 9 Years,"Black, Non-Hispanic",Missing,Missing,Missing,Missing
2,2020/06/14,2020/06/28,NaN,2020/06/14,Laboratory-confirmed case,Unknown,0 - 9 Years,"Black, Non-Hispanic",No,Missing,No,Missing
3,2020/11/09,2020/11/11,NaN,2020/11/09,Laboratory-confirmed case,Unknown,0 - 9 Years,"Black, Non-Hispanic",No,Missing,No,Missing
4,2020/11/28,2020/11/28,NaN,NaN,Laboratory-confirmed case,Unknown,0 - 9 Years,"Black, Non-Hispanic",Missing,Missing,Missing,Missing


In [5]:
# check how many rows and columns
covid.shape

(24441351, 12)

In [6]:
# look at the different columns/attributes
covid.columns.values.tolist()

['cdc_case_earliest_dt ',
 'cdc_report_dt',
 'pos_spec_dt',
 'onset_dt',
 'current_status',
 'sex',
 'age_group',
 'race_ethnicity_combined',
 'hosp_yn',
 'icu_yn',
 'death_yn',
 'medcond_yn']

In [7]:
# subset the dataframe into only desired columns
covid = covid[['cdc_case_earliest_dt ', 'sex', 'age_group', 'race_ethnicity_combined', 'hosp_yn', 'icu_yn', 'death_yn', 'medcond_yn']]

In [8]:
# initiate offline mode for plotly and cufflinks to create interactive plots
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [9]:
# create new count column to count the number of cases
covid['count'] = 1

In [10]:
# create a dataframe to hold only the cases resulting in deaths
deaths = covid[covid['death_yn'] == 'Yes']

In [11]:
# see how many deaths
deaths.shape

(445681, 9)

# Case and Death Demographics

In [12]:
# remove cases with missing age group information
covid = covid[covid['age_group'] != 'Missing']
deaths = deaths[deaths['age_group'] != 'Missing']

In [22]:
# create new dataframes grouped by age groups with counts
covid_age = covid.groupby('age_group').count()[['count']]
deaths_age = deaths.groupby('age_group').count()[['count']]

In [23]:
covid_age.head()

,count
age_group,
0 - 9 Years,1125623
10 - 19 Years,2675462
20 - 29 Years,4560168
30 - 39 Years,3977849
40 - 49 Years,3612385


In [24]:
# create new age columns using index values
covid_age['age'] = covid_age.index
deaths_age['age'] = deaths_age.index

In [25]:
# create ages list holding population values for each age group using US Census 2019 estimates
ages = [20331969, 21363868, 23069322, 22239551, 19992494, 20729100, 18056503, 10818305, 5056212]

In [26]:
# create new population columns using values in ages list
covid_age['population'] = ages
deaths_age['population'] = ages

In [27]:
# create new population percentage columns calculated by dividing individual values by totals
covid_age['Percentage of Population'] = round((covid_age['population'] / covid_age['population'].sum()) * 100, 2)
deaths_age['Percentage of Population'] = round((deaths_age['population'] / deaths_age['population'].sum()) * 100, 2)

In [28]:
# create new case and death percentage columns calculated by dividing individual values by totals
covid_age['Percentage of Cases'] = round((covid_age['count'] / covid_age['count'].sum()) * 100, 2)
deaths_age['Percentage of Deaths'] = round((deaths_age['count'] / deaths_age['count'].sum()) * 100, 2)

In [29]:
# merge case and death dataframes together into combined dataframe
comb_age = pd.merge(left=covid_age, right=deaths_age, how='left', on=['age', 'population', 'Percentage of Population'])

In [30]:
comb_age.head()

,count_x,age,population,Percentage of Population,Percentage of Cases,count_y,Percentage of Deaths
0,1125623,0 - 9 Years,20331969,12.58,4.64,208,0.05
1,2675462,10 - 19 Years,21363868,13.22,11.02,421,0.09
2,4560168,20 - 29 Years,23069322,14.27,18.79,2102,0.47
3,3977849,30 - 39 Years,22239551,13.76,16.39,5230,1.17
4,3612385,40 - 49 Years,19992494,12.37,14.88,12787,2.87


In [31]:
# create interactive bar plot showing case, death, and population percentages by age group
comb_age.iplot(kind='bar', x ='age', y =['Percentage of Cases', 'Percentage of Deaths', 'Percentage of Population'], title='COVID-19 Cases and Deaths by Age Group')

In [32]:
# remove cases with missing, other, and unknown listed in sex column
covid = covid[(covid['sex'] != 'Missing') & (covid['sex'] != 'Other') & (covid['sex'] != 'Unknown')]
deaths = deaths[(deaths['sex'] != 'Missing') & (deaths['sex'] != 'Other') & (deaths['sex'] != 'Unknown')]

In [33]:
# create new dataframes grouped by sex with counts
covid_sex = covid.groupby('sex').count()[['count']]
deaths_sex = deaths.groupby('sex').count()[['count']]

In [34]:
# create new gender columns using index values
covid_sex['gender'] = covid_sex.index
deaths_sex['gender'] = deaths_sex.index

In [35]:
# create sexes list holding population values for each sex using US Census 2019 estimates
sexes = [166582199, 161657324]

In [36]:
# create new population columns using values in sexes list
covid_sex['population'] = sexes
deaths_sex['population'] = sexes

In [37]:
# create new population percentage columns calculated by dividing individual values by totals
covid_sex['Percentage of Population'] = round((covid_sex['population'] / covid_sex['population'].sum()) * 100, 2)
deaths_sex['Percentage of Population'] = round((deaths_sex['population'] / deaths_sex['population'].sum()) * 100, 2)

In [38]:
# create new case and death percentage columns calculated by dividing individual values by totals
covid_sex['Percentage of Cases'] = round((covid_sex['count'] / covid_sex['count'].sum()) * 100, 2)
deaths_sex['Percentage of Deaths'] = round((deaths_sex['count'] / deaths_sex['count'].sum()) * 100, 2)

In [39]:
# merge case and death dataframes together into combined dataframe
comb_sex = pd.merge(left=covid_sex, right=deaths_sex, how='left', on=['gender', 'population', 'Percentage of Population'])

In [40]:
# create interactive bar plot showing case, death, and population percentages by gender
comb_sex.iplot(kind='bar', x ='gender', y =['Percentage of Cases', 'Percentage of Deaths', 'Percentage of Population'], title='COVID-19 Cases and Deaths by Gender')

In [41]:
# remove cases with missing and unknown listed in race column
covid = covid[(covid['race_ethnicity_combined'] != 'Missing') & (covid['race_ethnicity_combined'] != 'Unknown')]
deaths = deaths[(deaths['race_ethnicity_combined'] != 'Missing') & (deaths['race_ethnicity_combined'] != 'Unknown')]

In [42]:
# create new dataframes grouped by race with counts
covid_race = covid.groupby('race_ethnicity_combined').count()[['count']]
deaths_race = deaths.groupby('race_ethnicity_combined').count()[['count']]

In [43]:
# create new race columns using index values
covid_race['race'] = covid_race.index
deaths_race['race'] = deaths_race.index

In [44]:
# create races list holding population percentage values for each race using US Census 2019 estimates
races = [1.3, 5.9, 13.4, 18.5, 2.8, 0.2, 60.1]

In [45]:
# create new population columns using values in races list
covid_race['Percentage of Population'] = races
deaths_race['Percentage of Population'] = races

In [46]:
# create new case and death percentage columns calculated by dividing individual values by totals
covid_race['Percentage of Cases'] = round((covid_race['count'] / covid_race['count'].sum()) * 100, 2)
deaths_race['Percentage of Deaths'] = round((deaths_race['count'] / deaths_race['count'].sum()) * 100, 2)

In [47]:
# merge case and death dataframes together into combined dataframe
comb_race = pd.merge(left=covid_race, right=deaths_race, how='left', on=['race', 'Percentage of Population'])

In [48]:
comb_race.head()

,count_x,race,Percentage of Population,Percentage of Cases,count_y,Percentage of Deaths
0,156299,"American Indian/Alaska Native, Non-Hispanic",1.3,0.98,3378,0.89
1,526076,"Asian, Non-Hispanic",5.9,3.31,15025,3.97
2,1752294,"Black, Non-Hispanic",13.4,11.03,51441,13.58
3,4484617,Hispanic/Latino,18.5,28.24,69991,18.48
4,967792,"Multiple/Other, Non-Hispanic",2.8,6.09,15732,4.15


In [49]:
# create interactive bar plot showing case, death, and population percentages by race
comb_race.iplot(kind='bar', x ='race', y =['Percentage of Cases', 'Percentage of Deaths', 'Percentage of Population'], title='COVID-19 Cases and Deaths by Race')

# Cases and Deaths over Time

In [50]:
# create new dataframes of combined US daily values grouped by date with counts 
covid_date = covid.groupby('cdc_case_earliest_dt ').count()[['count']]
deaths_date = deaths.groupby('cdc_case_earliest_dt ').count()[['count']]

In [51]:
# create new date columns using index values
covid_date['date'] = covid_date.index
deaths_date['date'] = deaths_date.index

In [83]:
covid_date.head()

,count,date
cdc_case_earliest_dt,,
2020/01/01,601,2020/01/01
2020/01/02,177,2020/01/02
2020/01/03,141,2020/01/03
2020/01/04,199,2020/01/04
2020/01/05,198,2020/01/05


In [52]:
# create interactive bar plot showing number of cases by date
covid_date.iplot(kind='bar', x ='date', y ='count', title='United States Daily COVID-19 Cases')

In [53]:
# create interactive bar plot showing number of deaths by date
deaths_date.iplot(kind='bar', x ='date', y ='count', title='United States Daily COVID-19 Deaths')

In [54]:
# create interactive line plot showing number of cases by date
covid_date.iplot(kind='lines', x ='date', y ='count', title='United States Daily COVID-19 Cases')

In [55]:
# create interactive line plot showing number of deaths by date
deaths_date.iplot(kind='lines', x ='date', y ='count', title='United States Daily COVID-19 Deaths')

In [97]:
# create new dataframes grouped by date and age group with counts
covid_date_age = covid.groupby(['cdc_case_earliest_dt ', 'age_group']).count()[['count']]
deaths_date_age = deaths.groupby(['cdc_case_earliest_dt ', 'age_group']).count()[['count']]

In [98]:
# reset the indices for plotting columns
covid_date_age = covid_date_age.reset_index()
deaths_date_age = deaths_date_age.reset_index()

In [99]:
covid_date_age['cdc_case_earliest_dt ']=pd.to_datetime(covid_date_age['cdc_case_earliest_dt '])
deaths_date_age['cdc_case_earliest_dt ']=pd.to_datetime(deaths_date_age['cdc_case_earliest_dt '])

In [100]:
covid_date_age.head()

,cdc_case_earliest_dt,age_group,count
0,2020-01-01,0 - 9 Years,5
1,2020-01-01,10 - 19 Years,15
2,2020-01-01,20 - 29 Years,28
3,2020-01-01,30 - 39 Years,39
4,2020-01-01,40 - 49 Years,49


In [101]:
# create interactive line plot showing number of cases by date and categorized by age group
covid_date_age.iplot(mode='lines', x='cdc_case_earliest_dt ', y='count', categories='age_group', title='United States Daily COVID-19 Cases by Age Group')

In [102]:
# create interactive line plot showing number of deaths by date and categorized by age group
deaths_date_age.iplot(mode='lines', x='cdc_case_earliest_dt ', y='count', categories='age_group', title='United States Daily COVID-19 Deaths by Age Group')

In [103]:
# create new dataframes grouped by date and race with counts
covid_date_race = covid.groupby(['cdc_case_earliest_dt ', 'race_ethnicity_combined']).count()[['count']]
deaths_date_race = deaths.groupby(['cdc_case_earliest_dt ', 'race_ethnicity_combined']).count()[['count']]

In [104]:
# reset indices for plotting columns
covid_date_race = covid_date_race.reset_index()
deaths_date_race = deaths_date_race.reset_index()

In [105]:
covid_date_race['cdc_case_earliest_dt ']=pd.to_datetime(covid_date_race['cdc_case_earliest_dt '])
deaths_date_race['cdc_case_earliest_dt ']=pd.to_datetime(deaths_date_race['cdc_case_earliest_dt '])

In [106]:
# create interactive line plot showing number of cases by date and categorized by race
covid_date_race.iplot(mode='lines', x='cdc_case_earliest_dt ', y='count', categories='race_ethnicity_combined', title='United States Daily COVID-19 Cases by Race')

In [107]:
# create interactive line plot showing number of deaths by date and categorized by race
deaths_date_race.iplot(mode='lines', x='cdc_case_earliest_dt ', y='count', categories='race_ethnicity_combined', title='United States Daily COVID-19 Deaths by Race')

In [108]:
# create new dataframes grouped by date and sex with counts
covid_date_sex = covid.groupby(['cdc_case_earliest_dt ', 'sex']).count()[['count']]
deaths_date_sex = deaths.groupby(['cdc_case_earliest_dt ', 'sex']).count()[['count']]

In [109]:
# reset indices for plotting columns
covid_date_sex = covid_date_sex.reset_index()
deaths_date_sex = deaths_date_sex.reset_index()

In [110]:
covid_date_sex['cdc_case_earliest_dt ']=pd.to_datetime(covid_date_sex['cdc_case_earliest_dt '])
deaths_date_sex['cdc_case_earliest_dt ']=pd.to_datetime(deaths_date_sex['cdc_case_earliest_dt '])

In [111]:
# create interactive line plot showing number of cases by date and categorized by sex
covid_date_sex.iplot(mode='lines', x='cdc_case_earliest_dt ', y='count', categories='sex', title='United States Daily COVID-19 Cases by Gender')

In [112]:
# create interactive line plot showing number of deaths by date and categorized by sex
deaths_date_sex.iplot(mode='lines', x='cdc_case_earliest_dt ', y='count', categories='sex', title='United States Daily COVID-19 Deaths by Gender')

In [113]:
# create new dataframes grouped by sex, age group, and race with counts
covid_hierarchy = covid.groupby(['sex', 'age_group', 'race_ethnicity_combined']).count()[['count']]
deaths_hierarchy = deaths.groupby(['sex', 'age_group', 'race_ethnicity_combined']).count()[['count']]

In [114]:
covid_hierarchy.head()

count
sex    age_group   race_ethnicity_combined                            
Female 0 - 9 Years American Indian/Alaska Native, Non-Hispanic    4992
                   Asian, Non-Hispanic                           11479
                   Black, Non-Hispanic                           42392
                   Hispanic/Latino                              139608
                   Multiple/Other, Non-Hispanic                  27780

In [115]:
# reset indices for plotting columns
covid_hierarchy = covid_hierarchy.reset_index()
deaths_hierarchy = deaths_hierarchy.reset_index()

In [116]:
# create a treemap of case counts organized hierarchically by sex, age group, and race
fig = px.treemap(covid_hierarchy,
                 path=['sex', 'age_group', 'race_ethnicity_combined'],
                 values='count',
                 color='race_ethnicity_combined',
                 width=1000, height=700,
                 title='United States COVID-19 Cases Demographics',
                 )

fig.show()

In [117]:
# create a treemap of deaths counts organized hierarchically by sex, age group, and race
fig = px.treemap(deaths_hierarchy,
                 path=['sex', 'age_group', 'race_ethnicity_combined'],
                 values='count',
                 color='race_ethnicity_combined',
                 width=1000, height=700,
                 title='United States COVID-19 Deaths Demographics',
                 )

fig.show()